# 基于 Qwen1.5-0.5b 的 RAG 文档问答
## 项目介绍
本项目旨在利用大语言模型，解析和处理 MindSpore 设计概览 PDF 文档，构建一个智能问答系统。​该系统不仅展示了大语言模型在专业领域文档理解和信息提取方面的能力，同时也为开发者提供了一个实践范例，说明如何结合预训练模型与特定领域知识，打造高效的问答解决方案。

## Step 1：安装依赖包
本项目主要基于 MindSpore NLP 和 Mindspore 进行开发，除上述两个库外，需先安装相关依赖库


In [1]:
!pip install pymupdf sentence-transformers faiss-cpu newspaper3k lxml[html_clean] jieba==0.42.1

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 8.8 MB/s eta 0:00:0000:0100:01
  Using cached jieba3k-0.35.1-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 6.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 3.5 MB/s eta 0:00:0000:0100:01
  Using cached transformers-4.53.2-py3-none-any.whl (10.8 MB)
  Using cached transformers-4.53.1-py3-none-any.whl (10.8 MB)
  Using cached transformers-4.53.0-py3-none-any.whl (10.8 MB)
  Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)
  Using cached transformers-4.52.3-py3-none-any.whl (10.5 MB)
  Using cached transforme

##  Step 2：RAG——加载 PDF 文档

#### RAG介绍
大语言模型在智能交互、语言理解等方面都展现出了强大能力，然而在实际业务场景中，通用的基础语言大模型往往难以满足特定的业务需求。
大模型的知识来源受限于其训练数据，目前主流的大模型，如ChatGPT、文心一言、通义千问等，其训练集主要基于网络公开数据。意味着它们往往难以获取实时、非公开或离线数据中的知识，从而限制了模型在某些专业领域的应用。
同时大模型可能面临幻觉问题，由于所有AI模型的底层原理都基于数学概率，大模型的输出实质上是一系列数值运算的结果。在某些情况下，这可能导致模型在不擅长的场景或缺乏相关知识时产生误导性的回答。这种幻觉问题的识别需要使用者具备相应领域的知识，从而限制了使用的效果。
再者，数据安全性也是现代社会关注的焦点。在利用大模型处理企业数据时，数据泄露的风险不容忽视。因此，许多企业或者个人用户不愿将私域数据上传至第三方平台进行训练，这在一定程度上限制了通用大模型在实际业务中的应用。

为了解决上述问题，检索增强生成（Retrieval Augmented Generation ，RAG）技术应运而生。
RAG技术结合了信息检索与自然语言生成，通过从大量文档中检索相关信息来增强大模型的生成能力。在实际操作中，RAG系统会先依据用户的查询需求，在庞大的知识库中迅速定位并检索到高度相关的文档。随后，系统会精心提炼这些文档中的宝贵知识，将其巧妙地融入用户输入文本之中，一同传递给大模型。这一过程不仅为大模型提供了丰富的可参考背景知识，还大幅提升了生成内容的准确性和针对性。

RAG的核心架构可以概括为“数据检索+智能生成”。在“数据检索”环节，依赖向量数据库技术，利用其高效的数据存储与检索机制，迅速锁定目标知识；而在“智能生成”阶段，则借助大模型和精细的提示工程技术，将检索到的知识精准利用，最终生成符合需求的答案。

项目使用的示例文档可通过下面代码下载并且加载。

In [2]:
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/white_paper/MindSpore_white_paperV1.1.pdf"

# 下载 MindSpore 设计概览 PDF 文档
download(url, "MindSpore_Design_Overview.pdf", replace=True)


file_sizes: 100%|██████████████████████████| 1.83M/1.83M [00:00<00:00, 4.30MB/s]
Successfully downloaded file to MindSpore_Design_Overview.pdf


'MindSpore_Design_Overview.pdf'

In [3]:
import fitz  # PyMuPDF
pdf_path = 'MindSpore_Design_Overview.pdf'  # 需提前下载该文档

doc = fitz.open(pdf_path)
pdf_text = [page.get_text() for page in doc]

## Step 3：清洗 PDF 文本

In [4]:
import pandas as pd
def clean_text(text):
    return ' '.join(text.split())
pdf_df = pd.DataFrame({
    'page': list(range(1, len(pdf_text)+1)),
    'text': [clean_text(t) for t in pdf_text]
})

##  Step 4：切分为语义块

In [5]:
chunks = []
for _, row in pdf_df.iterrows():
    text = row['text']
    for i in range(0, len(text), 300):
        chunk = text[i:i+300]
        chunks.append(chunk)

##  Step 5：向量化并构建 FAISS 检索器

由于一般情况下加载的文档字符较多，常常会达到千位万位的数量级，如果大语言模型直接处理整个文档，可能会在分析过长的文本时遇到困难，难以准确抓取所需信息。为有效应对这一挑战，可以需要采取分割策略，将庞大的文档细化为若干小文本块。如此一来，在需要时，我们便可以灵活地调用相关文档片段，从而提升信息处理的准确性和效率。本课题中也将文档切成包含更小数量级的段落，并且每个段落间也有部分字符重叠，确保一个观点的相关背景信息不会因为切分被遗漏。

向量化是一个将文本数据转化为向量矩阵的过程，将每个文本块的内容换成向量，并将这些向量存储在向量数据库中，从而实现索引，方便程序在运行时可以快速检索到相关那内容。向量化的模型众多，Openai的ChatGPT-Embedding、百度的ERNIE-Embedding V1以及北京智源人工智能研究院的BGE模型都是目前主流的开源转换模型。

数据向量化后，构建索引并将其写入数据库的过程，简称为“数据入库”。在RAG场景下，有多种数据库可供选择，例如FAISS、Chromadb、Elasticsearch以及Milvus等。在选择适合的数据库时，应全面考虑业务的具体需求、硬件配置以及性能要求等诸多因素，以确保选出最适合的数据库方案。本项目采用较通用常见的FAISS数据库进行数据入库操作。


In [ ]:
### 建议下载到本地部署，比较快，可以从魔塔下载或者昇思平台下载
!git lfs install
!git clone https://www.modelscope.cn/Ceceliachenen/paraphrase-multilingual-MiniLM-L12-v2.git

In [7]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embed_model = SentenceTransformer('./paraphrase-multilingual-MiniLM-L12-v2')
chunk_embeddings = embed_model.encode(chunks)

index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(np.array(chunk_embeddings))

##  Step 6：定义检索接口

In [8]:
def query_pdf(query, top_k=3):
    q_vec = embed_model.encode([query])
    D, I = index.search(np.array(q_vec), top_k)
    return [chunks[i] for i in I[0]]

##  Step 7：加载Qwen1.5-0.5b模型
本项目使用小规模蒸馏模型

基于Mindspore框架和MindSpore NLP库开发安装还是很方便的

In [9]:
import mindspore
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindnlp.transformers import TextIteratorStreamer
from threading import Thread

# 加载tokenizer和模型
print("正在加载模型...")
tokenizer = AutoTokenizer.from_pretrained("/home/HwHiAiUser/Qwen1.5-0.5B-Chat", mirror="modelers", ms_dtype=mindspore.float16)
model = AutoModelForCausalLM.from_pretrained("/home/HwHiAiUser/Qwen1.5-0.5B-Chat", mirror="modelers", ms_dtype=mindspore.float16)
print("模型加载完成！")


[WARNING] ME(199412:255086666027040,MainProcess):2025-08-16-21:30:04.674.500 [mindspore/context.py:1402] For 'context.set_context', the parameter 'ascend_config' will be deprecated and removed in a future version. Please use the api mindspore.device_context.ascend.op_precision.precision_mode(),
                                                       mindspore.device_context.ascend.op_precision.op_precision_mode(),
                                                       mindspore.device_context.ascend.op_precision.matmul_allow_hf32(),
                                                       mindspore.device_context.ascend.op_precision.conv_allow_hf32(),
                                                       mindspore.device_context.ascend.op_tuning.op_compile() instead.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.482 seconds.
Prefix dict has been built successfully.


正在加载模型...


Qwen2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


模型加载完成！


##  Step 8：构建回答的流式输出函数

In [10]:
system_prompt = "你是个能根据文档内容回答问题的专家，需要根据用户提供的文档内容来具体解答"
def chat_with_model(user_input):
    """与模型进行单轮对话，支持流式输出"""
    # 构建消息格式
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_input}
    ]
    
    # 应用聊天模板
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="ms",
        tokenize=True
    )
    
    # 创建流式输出器
    streamer = TextIteratorStreamer(tokenizer, timeout=300, skip_prompt=True, skip_special_tokens=True)
    
    # 生成参数
    generate_kwargs = dict(
        input_ids=input_ids,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.9,
        temperature=0.1,
        num_beams=1,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # 在单独线程中启动生成
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    
    # 流式输出tokens
    full_response = ""
    for new_token in streamer:
        if '</s>' in new_token:  # 检查停止token
            break
        print(new_token, end="", flush=True)
        full_response += new_token
    
    return full_response

##  Step 9：测试模型结合文档回答的效果

In [12]:
query="什么是MindSpore？具体介绍一下"
retrieved = "\n".join(query_pdf(query))
chat_with_model(f"以下是相关资料：{retrieved}\n请回答：{query}")

MindSpore是一种全新的深度学习计算框架，旨在实现易开发、高效执行和全场景覆盖三大目标。MindSpore 由几个主要组件组成：MindExpression（ME）、Mind Compiler（MC）、MindData（MD）、MindRE 和 MindArmour（MA）。其中，MindExpression 是一个用于构建和运行深度学习模型的中间表达，而 MindCompiler 是用于将源代码转换为可执行的机器代码的工具，MindData 是用于存储和管理数据的框架，MindRE 是用于构建和运行深度学习模型的中间表达，MindArmour 是用于构建和运行深度学习模型的中间表达。